In [1]:
import pandas as pd
import plotly.express as px

In [31]:
df = pd.read_parquet("../src/logs/log_ben_multiplication.parquet")

In [32]:
df.head()

,a,op,b,answers,result,success,failure,session_datetime,weights,mask,weights_mask,total_time_spent,time_spent_per_op
78,7,*,7,49,49,1,0,2023-04-30 18:07:53.094336-04:00,1.0,1.0,1.0,7.987035,1.597407
84,1,*,8,8,8,1,0,2023-04-30 18:07:53.094336-04:00,1.0,1.0,1.0,7.987035,1.597407
15,4,*,2,8,8,1,0,2023-04-30 18:07:53.094336-04:00,1.0,1.0,1.0,7.987035,1.597407
12,1,*,2,2,2,1,0,2023-04-30 18:07:53.094336-04:00,1.0,1.0,1.0,7.987035,1.597407
68,9,*,6,12,54,0,1,2023-04-30 18:07:53.094336-04:00,1.0,1.0,1.0,7.987035,1.597407


In [37]:
def calculate_score(df: pd.DataFrame) -> pd.DataFrame:
    df_sums = df.groupby("session_datetime")[["success", "failure"]].sum()
    df_sums["total"] = df_sums["success"] + df_sums["failure"]
    df_sums["pct_success"] = df_sums["success"] / df_sums["total"]

    df_last = df.groupby("session_datetime")["time_spent_per_op"].last().to_frame()
    
    df_res = df_last.join(df_sums[["pct_success"]], how="left")
    df_res["score"] = 100.0 * df_res["pct_success"] / df_res["time_spent_per_op"]
    df_res["index"] = range(1, len(df_res)+1)
    return df_res.sort_index(ascending=True)

In [38]:
df_res = calculate_score(df)
df_res

,time_spent_per_op,pct_success,score,index
session_datetime,,,,
2023-04-30 18:07:53.094336-04:00,1.597407,0.800000,50.081166,1
2023-04-30 18:29:00.776024-04:00,1.690875,1.000000,59.140963,2
2023-04-30 23:29:12.864922-04:00,2.765943,1.000000,36.154041,3
2023-04-30 23:29:40.987023-04:00,2.585463,0.666667,25.785195,4
2023-04-30 23:30:59.926627-04:00,1.939097,0.500000,25.785195,5
2023-04-30 23:41:34.703667-04:00,2.442560,1.000000,40.940653,6
2023-04-30 23:42:05.281449-04:00,3.159421,1.000000,31.651368,7
2023-04-30 23:42:27.868743-04:00,3.159421,0.500000,15.825684,8
2023-04-30 23:43:25.006781-04:00,3.159421,1.000000,31.651368,9


In [39]:
px.line(df_res, x="index", y="score", height=500, width=1000)